In [ ]:
import oci
import requests
import json
import uuid
from contextlib import closing
from requests import Request, Response

class AuthHelper:
    """
    AuthHelper allows creating an OCI signer with either API key or security_token (which are short term sessions)
    """
    def __init__(self, oci_config_path:str, oci_profile: str, use_security_token:bool = False):
        config = oci.config.from_file(file_location=oci_config_path,profile_name=oci_profile)
        if use_security_token:
            with open(config["security_token_file"], 'r') as f:
                 token = f.read()
            private_key = oci.signer.load_private_key_from_file(config["key_file"])            
            self.signer = oci.auth.signers.SecurityTokenSigner(token, private_key)
        else:
            self.signer = oci.signer.Signer(
                tenancy=config["tenancy"],
                user=config["user"],
                fingerprint=config["fingerprint"],
                private_key_file_location=config["key_file"],
                pass_phrase=config.get("pass_phrase")
            )

    @property
    def Signer(self):
        return self.signer

class MyRawJsonRpcClient:
    """
    Simple class using requests lib to post JSON to chat endpoint using OCI signing
    """
    def __init__(self, chat_url:str, oci_config_path:str, oci_profile: str, sessionKey:str, use_security_token:bool = True):
        self.authhelper = AuthHelper(oci_config_path=oci_config_path, oci_profile=oci_profile, use_security_token=use_security_token)
        self.authsigner = self.authhelper.Signer
        self.chat_url = chat_url
        self.sessionKey = sessionKey
    
    def send(self, input:str) -> Response:
        body = {
            "isStreamEnabled" : False,
            "sessionKey" : self.sessionKey,
            "trace" : False,
            "input" :[{
                "role":"User",
                "content":[{
                    "type" : "INPUT_TEXT",
                    "text" : input                   
                }]
            }]
        }

        response:Request = requests.post(
            url =self.chat_url,
            params = None,
            auth = self.authsigner,
            json=body,
            headers={}
        )
        return response

In [ ]:
# create a session ID using the current timestamp. Could be anything. 
# Re-use the same sessionKey if you want to resume an existing session. 
import datetime
now = str(datetime.datetime.now())
sessionKey=f"triggersession-{now}"

In [ ]:
# create a client instance to send a message to the agent flow:
# your config file should be stored in an AIDP volume with a path like: /Volumes/{catalog-name}/{schema-name}/{volume-name}/{file-name}
# Store the secret key alongside the config file in the same volume. Provide a full path to the secret file in the config file. 
# the oci_profile is usually DEFAULT but you may have other profiles defined in your config file. Choose the profile you want to use. 
client = MyRawJsonRpcClient(chat_url="<insert-your-agent-flow-chat-endpoint-here>",
                            oci_config_path="<your-oci-config-file-path>>",
                            oci_profile = "<profile-name-to-use>",
                            sessionKey= sessionKey,
                            use_security_token = False
                           )

In [ ]:
# defining a simple user input to send to the agent flow and sending it:
user_input = f"Hello, tell me a good dad joke."
r = client.send(input = user_input)
# response from the agent flow:
response_json = r.json()

# Optionally you can write the input and output to a file for later review in an AIDP volume: 
# A file path is : /Volumes/{catalog-name}/{schema-name}/{volume-name}/{file-name}
with open(f"<path-to-file>","w") as file:
    file.write(user_input)
    file.write("\n")
    file.write(json.dumps(response_json, indent=1))
    file.write("\n\n")
    file.close()